# Test for ragchecker

In [64]:
import os

from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from ragchecker.integrations.llama_index import response_to_rag_results
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics

### Create a naive model

In [ ]:
! pip install llama-index-readers-web

In [11]:
from llama_index.readers.web import SimpleWebPageReader
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://www.ml.school/"]
)

In [13]:
from llama_index.core import SummaryIndex
index = SummaryIndex.from_documents(documents)

In [14]:
# Create query engine
rag_application = index.as_query_engine()

In [15]:
# User query and groud truth answer
user_query = "What is Price of this course?"
gt_answer = "The price of the course is $500"

In [16]:
# Get response from LlamaIndex
response_object = rag_application.query(user_query)

In [41]:
response_object

Response(response='$500', source_nodes=[NodeWithScore(node=TextNode(id_='e979c3f5-414f-4f54-b545-a5e073e5c0e5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='http://www.ml.school/', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='01d5d29cc37a267e7fd48034058b35d6aba880a539ad3e294c217e45b98a0413'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8c5d361b-18ba-4355-80ff-98356ad870f0', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='3442213380f08644e4288760e626113c4232b05faf77697904e43d5108da3ccf')}, text='> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck th

In [84]:
# Convert to RAGChecker format
rag_result = response_to_rag_results(
    query=user_query,
    gt_answer=gt_answer,
    response_object=response_object,
)

In [85]:
rag_result

{'query_id': None,
 'query': 'What is Price of this course?',
 'gt_answer': 'The price of the course is $500',
 'response': '$500',
 'retrieved_context': [{'text': '> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck the schedule for more details about upcoming cohorts.\n\nI want to join!Sign in\n\n## Learn how to **design** , **build** , **deploy** , and **scale** machine\nlearning systems to solve real-world problems.\n\nI\'ll lose my mind if I see another book or course teaching people the same\nbasic ideas for the hundredth time. Most people are stuck in beginner mode,\nand finding help to solve real-world problems is hard.\n\nI want to change that.\n\nI started writing software 30 years a

In [44]:
if rag_result.get('query_id') is None:
    rag_result['query_id'] = "001"

In [50]:
rag_result

{'query_id': '001',
 'query': 'What is Price of this course?',
 'gt_answer': 'The price of the course is $500',
 'response': '$500',
 'retrieved_context': [{'text': '> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck the schedule for more details about upcoming cohorts.\n\nI want to join!Sign in\n\n## Learn how to **design** , **build** , **deploy** , and **scale** machine\nlearning systems to solve real-world problems.\n\nI\'ll lose my mind if I see another book or course teaching people the same\nbasic ideas for the hundredth time. Most people are stuck in beginner mode,\nand finding help to solve real-world problems is hard.\n\nI want to change that.\n\nI started writing software 30 years 

In [86]:
# Create RAGResults object
rag_results = RAGResults.from_dict({"results": [rag_result]})

/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/site-packages/dataclasses_json/core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type query_id detected when decoding RAGResult.
  warnings.warn(


In [52]:
rag_results

RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)

In [88]:
with open("/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/eval/rag_response.json") as fp:
    rag_result2 = fp.read()

In [89]:
rag_result2

'{\n    "results": [\n        {\n            "query_id": "000",\n            "query": "What is Price of this course?",\n            "gt_answer": "The price of the course is $500",\n            "response": "$500",\n            "retrieved_context": [\n                {\n                    "doc_id": "d37b09b5-c690-476d-b194-6dbacc8c5e25",\n                    "text": "\\"This is the best machine learning course I\'ve done. Worth every cent.\\"\\n\\n**Jose Reyes** , AI/ML at Cevo Australia\\n\\n# Building Machine Learning Systems That Don\'t Suck\\n\\nA **live, interactive program** that\'ll help you build **production-ready** machine learning systems from the ground up.\\n\\nNext cohort: November 4 - 21, 2024\\n\\nCheck the schedule for more details."\n                },\n                {\n                    "doc_id": "62a75aad-a40b-4143-965b-753b092ab0d8",\n                    "text": "You\'ll come out with practical skills and insights into what it takes to build systems that work in

In [90]:
rag_results2 = RAGResults.from_json(rag_result2)

In [91]:
rag_results2

RAGResults(
  2 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)

In [97]:
import os
from refchecker import LLMExtractor, LLMChecker


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# GPT-4o from OpenAI
model = 'gpt-4o'

extractor = LLMExtractor(model=model, batch_size=8)
checker = LLMChecker(model=model, batch_size=8)

In [98]:
import litellm

# Set the API key for litellm (OpenAI API)
litellm.api_key = OPENAI_API_KEY


# Initialize RAGChecker
# Ensure no `max_workers` is being passed or used internally
evaluator = RAGChecker(
    extractor_name=extractor,
    checker_name=checker,
    batch_size_extractor=32,
    batch_size_checker=32
)

# Evaluate using RAGChecker
evaluator.evaluate(rag_results2, all_metrics)

# Print detailed results
print(rag_results)

2024-10-10 17:19:17.768 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 2 RAG results.
  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'LLMExtractor' object has no attribute 'split'

In [95]:
from litellm import completion

model = "gpt-4-1106-preview"  # or "gpt-4-0125-preview" for the latest version

# Initialize RAGChecker
evaluator = RAGChecker(
    extractor=lambda prompt: completion(model=model, messages=[{"role": "user", "content": prompt}]),
    checker=lambda prompt: completion(model=model, messages=[{"role": "user", "content": prompt}]),
    batch_size_extractor=32,
    batch_size_checker=32,
)

# Evaluate using RAGChecker
evaluator.evaluate(rag_results, all_metrics)

# Print detailed results
print(rag_results)

2024-10-10 17:10:25.030 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for response of 1 RAG results.
  0%|          | 0/1 [00:00<?, ?it/s]

litellm.APIConnectionError: Object of type function is not JSON serializable
Traceback (most recent call last):
  File "/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/site-packages/litellm/main.py", line 2522, in completion
    response = bedrock_converse_chat_completion.completion(
  File "/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/site-packages/litellm/llms/bedrock/chat/converse_handler.py", line 308, in completion
    data = json.dumps(_data)
  File "/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/ragchecker/venv/lib/python3.10/json/encoder.py", line 257, in iteren

  0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

### It is confirmed that litellm worked with the OPENAI API KEY

In [56]:
import os 
from litellm import completion

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# openai call
response = completion(
    model = "gpt-4o", 
    messages=[{ "content": "Hello, how are you?","role": "user"}]
)

In [57]:
response

ModelResponse(id='chatcmpl-AGwuiekQC9zqLv5XdghBOKqPIy1b8', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", role='assistant', tool_calls=None, function_call=None))], created=1728603076, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_6b68a8204b', usage=Usage(completion_tokens=30, prompt_tokens=13, total_tokens=43, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, text_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)), service_tier=None)